In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import cv2 as cv

import os, gc, csv

In [2]:
import torch

In [3]:
gpu = torch.cuda.is_available()
device = torch.device("cpu")

if gpu:
    print("use gpu")
    #device = torch.device("cuda:0")

use gpu


In [4]:
res_unet = torch.load('E:/Workspace/ke30_u7_AASCE2019/resunet_mean.pth')

net = res_unet.to(device)

In [5]:
class BoostNetChallgeDataset(torch.utils.data.Dataset):
    def __init__(self, names, transform=None):
        self.images = []
        self.labels = []
          
        for i, path in enumerate(names):
            image_name = path.name
            self.labels.append(image_name)
            origin_image = cv.imread(str(path),0)
            target_height = 512
            target_width = 256
            image = np.zeros((target_height, target_width), np.uint8)
            cv.resize(origin_image, (target_width, target_height), image)
            
            image = np.reshape(image, (1, image.shape[0], image.shape[1]))
            image_tensor = torch.from_numpy(image).float()
            
            self.images.append(image_tensor)
 
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label
    
    def __len__(self):
        return len(self.images)

In [6]:
# zero the parameter gradients
#optimizer.zero_grad()
from pathlib import Path
p = Path('.')
test_images = list(p.glob('./test/*.jpg'))
len(test_images)

98

In [7]:
target_data = BoostNetChallgeDataset(test_images)
  
batch = 4
target_loader = torch.utils.data.DataLoader(target_data, batch_size=batch,
                                         shuffle=True, num_workers=0)

print("number of batches(testing): ",len(target_loader))

number of batches(testing):  25


In [8]:
for path in test_images:
  image_name = path.name
  origin_image = cv.imread(str(path),0)
  target_height = 512
  target_width = 256
  image = np.zeros((target_height, target_width), np.uint8)
  cv.resize(origin_image, (target_width, target_height), image)
  dest = "E:/Workspace/ke30_u7_AASCE2019/test/resized/" + image_name
  print(dest)
  print(image.shape)
  cv.imwrite(dest, image)

E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-1.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-10.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-11.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-12.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-13.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-14.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-15.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-16.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-17.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-18.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-19.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-2.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2019/test/resized/01-July-2019-20.jpg
(512, 256)
E:/Workspace/ke30_u7_AASCE2

In [ ]:
#dataiter = iter(target_loader)
land_pred = np.empty([98, 136])
image_names = []
  
index = 0
for images, labels in target_loader:
  batch, channel, height, width = images.shape
  ret = net(images.to(device))
  for i in range(batch):
    predict = ret[i]
    lineIndex = index + i;
    image_names.append(labels[i])
    #print(image_names)
    point_num = len(predict)
    #print("copy data for line : " + str(lineIndex+ 1))
    for j in range(point_num):
      land_pred[lineIndex,j]= predict[j]
      
  index+=4

C:\Users\wlgq\AppData\Local\conda\conda\envs\pt\lib\site-packages\torch\nn\functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [ ]:
np.savetxt("E:/Workspace/ke30_u7_AASCE2019/test/landmarks.csv", land_pred, delimiter=",", fmt="%1.5f")

In [ ]:
with open("E:/Workspace/ke30_u7_AASCE2019/test/filenames.csv",'w',newline='') as f:
    wr = csv.writer(f)
    for name in image_names:
      wr.writerow([name,])

## Generate mat files

In [ ]:
import mat4py
land_array = []
images=[]
p2 = []
width = 256
height = 512
mat_out = 'E:/Workspace/ke30_u7_AASCE2019/test/matfiles'

In [ ]:
landmarks = csv.reader(open('E:/Workspace/ke30_u7_AASCE2019/test/landmarks.csv', 'r'))
for i, land in enumerate(landmarks):
       land_array.append(land)
len(land_array)

In [ ]:
for i, name in enumerate(images):
    point_num = len(land_array[i]) // 2
    predict = land_array[i]
    points = np.empty((point_num,2))
    for j in range(point_num):
        left = int(float(predict[j]) * width)
        right = int(float(predict[j + point_num])* height)
        points[j] = [left,right]
                    
    data = {'p2' : points.tolist()}
    filename = mat_out +"/"+ name[0] +".mat"
    mat4py.savemat(filename, data)